### CONTEXT DATA ANALYSIS

In [213]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

Get the Data from the official source (INE)

- [Ocupados por sexo y rama de actividad](https://www.ine.es/jaxiT3/Tabla.htm?t=4128)
- [Salario por sexo y rama de actividad](https://www.ine.es/jaxiT3/Tabla.htm?t=28185)

In [265]:
# OCUPADOS data loading
df_ocupados = pd.read_csv("ocupados_por_sexo_y_rama_de_actividad.csv", sep = ";", decimal = ",")
df_ocupados['Total'] = pd.to_numeric(df_ocupados['Total'].apply(lambda numstr: numstr.replace(".", "").replace(",", ".")))
df_ocupados.rename(columns = {"Rama de actividad CNAE 2009" : "Grupo CNAE 2009"}, inplace = True)
df_ocupados = df_ocupados.loc[df_ocupados['Unidad'] == "Porcentaje", ["Grupo CNAE 2009", "Sexo", "Periodo", "Total"]]
df_ocupados.info()
df_ocupados.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 792 entries, 0 to 791
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Grupo CNAE 2009  792 non-null    object 
 1   Sexo             792 non-null    object 
 2   Periodo          792 non-null    object 
 3   Total            791 non-null    float64
dtypes: float64(1), object(3)
memory usage: 30.9+ KB


,Grupo CNAE 2009,Sexo,Periodo,Total
0,B_S Todas las secciones,Ambos sexos,2019T4,100.0
1,B_S Todas las secciones,Ambos sexos,2019T3,100.0
2,B_S Todas las secciones,Ambos sexos,2019T2,100.0
3,B_S Todas las secciones,Ambos sexos,2019T1,100.0
4,B_S Todas las secciones,Ambos sexos,2018T4,100.0


In [266]:
# OCUPPADOS data loading and casting
df_ocupados['Periodo'] = df_ocupados['Periodo'].apply(lambda strperiod: strperiod[:-2])
df_ocupados['Periodo'] = df_ocupados['Periodo'].astype("int")
df_ocupados = pd.DataFrame(df_ocupados.groupby(['Grupo CNAE 2009', "Sexo", "Periodo"])['Total'].mean(), columns = ['Total'])
df_ocupados.info()
df_ocupados.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 198 entries, ('A Agricultura, ganadería, silvicultura y pesca', 'Ambos sexos', 2017) to ('U Actividades de organizaciones y organismos extraterritoriales', 'Mujeres', 2019)
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Total   198 non-null    float64
dtypes: float64(1)
memory usage: 2.5+ KB


Total
Grupo CNAE 2009                                Sexo        Periodo       
A Agricultura, ganadería, silvicultura y pesca Ambos sexos 2017      4.35
                                                           2018      4.20
                                                           2019      4.05
                                               Hombres     2017      6.05
                                                           2018      5.90

In [267]:
# SALARIO data loading and casting
df_salario = pd.read_csv("salario_medio_por_sexo_y_rama_de_actividad.csv", sep = ";", decimal = ",")
df_salario['Total'] = pd.to_numeric(df_salario['Total'].apply(lambda numstr: numstr.replace(".", "").replace(",", ".")))
df_salario['Periodo'] = df_salario['Periodo'].astype("int")
df_salario.rename(columns = {"Secciones de la CNAE 2009" : "Grupo CNAE 2009", "Total": "Salario"}, inplace = True)
df_salario = df_salario.set_index(['Grupo CNAE 2009', 'Sexo', "Periodo"])
df_salario.info()
df_salario.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 171 entries, ('B_S Todas las secciones', 'Ambos sexos', 2019) to ('S Otros servicios', 'Hombres', 2017)
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Salario  171 non-null    float64
dtypes: float64(1)
memory usage: 3.0+ KB


Salario
Grupo CNAE 2009         Sexo        Periodo          
B_S Todas las secciones Ambos sexos 2019     24395.98
                                    2018     24009.12
                                    2017     23646.50
                        Mujeres     2019     21682.02
                                    2018     21011.89

**LEFT JOIN OF BOTH TABLES**

Next table reflects the **spanish laboral market** context between 2017 and 2019, by giving information about occupation weight and mean salary splitted by activity group (CMAE 2009 classification) and gender.

By looking at the statistics, we can immediately identify the presence of a gender gap. It's extreamly severe when comparing `salaries`: **by 2017, woman got paid an average of 5783.99 eur less than man. By 2019, this difference got reduced to 5252.36 eur.**  

We can also see a difference in gender presence by activity group.

In [275]:
# Proceed with left (outer) join
df = df_ocupados.join(df_salario, how = "left")
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 198 entries, ('A Agricultura, ganadería, silvicultura y pesca', 'Ambos sexos', 2017) to ('U Actividades de organizaciones y organismos extraterritoriales', 'Mujeres', 2019)
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Total    198 non-null    float64
 1   Salario  171 non-null    float64
dtypes: float64(2)
memory usage: 12.2+ KB


In [280]:
pd.set_option('display.max_rows', None)
df

Total  \
Grupo CNAE 2009                                    Sexo        Periodo            
A Agricultura, ganadería, silvicultura y pesca     Ambos sexos 2017       4.350   
                                                               2018       4.200   
                                                               2019       4.050   
                                                   Hombres     2017       6.050   
                                                               2018       5.900   
                                                               2019       5.700   
                                                   Mujeres     2017       2.300   
                                                               2018       2.175   
                                                               2019       2.050   
B Industrias extractivas                           Ambos sexos 2017       0.200   
                                                               2018       0.200   
                                                               2019       0.200   
                                                   Hombres     2017       0.300   
                                                               2018       0.300   
                                                               2019       0.300   
                                                   Mujeres     2017       0.000   
                                                               2018       0.000   
                                                               2019       0.050   
B_S Todas las secciones                            Ambos sexos 2017     100.000   
                                                               2018     100.000   
                                                               2019     100.000   
                                                   Hombres     2017     100.000   
                                                               2018     100.000   
                                                               2019     100.000   
                                                   Mujeres     2017     100.000   
                                                               2018     100.000   
                                                               2019     100.000   
C Industria manufacturera                          Ambos sexos 2017      12.700   
                                                               2018      12.625   
                                                               2019      12.625   
                                                   Hombres     2017      17.300   
                                                               2018      17.075   
                                                               2019      16.900   
                                                   Mujeres     2017       7.200   
                                                               2018       7.350   
                                                               2019       7.550   
D Suministro de energía eléctrica, gas, vapor y... Ambos sexos 2017       0.450   
                                                               2018       0.400   
                                                               2019       0.500   
                                                   Hombres     2017       0.650   
                                                               2018       0.550   
                                                               2019       0.600   
                                                   Mujeres     2017       0.200   
                                                               2018       0.250   
                                                               2019       0.275   
E Suministro de agua, actividades de saneamient... Ambos sexos 2017       0.750   
                                                               2018       0.775   
       

1. **Activity group compared by occupation**



2. **Activity groups compared by salary**

3. **Gender gap facts**

In [217]:
A = set(list(df_ocupados['Grupo CNAE 2009'].unique()))
B = set(list(df_salario['Grupo CNAE 2009'].unique()))

A - B

{'A Agricultura, ganadería, silvicultura y pesca',
 'T Actividades de los hogares como empleadores de personal doméstico, actividades de los hogares como productores de bienes y servicios para uso propio',
 'U Actividades de organizaciones y organismos extraterritoriales'}

In [145]:
# DATA MERGING

,Rama de actividad CNAE 2009,Sexo,Unidad,Periodo,Total
0,N Actividades administrativas y servicios auxi...,Ambos sexos,Valor absoluto,2022T1,1061.5
1,N Actividades administrativas y servicios auxi...,Ambos sexos,Valor absoluto,2021T4,1098.9
2,N Actividades administrativas y servicios auxi...,Ambos sexos,Valor absoluto,2021T3,1103.7
3,N Actividades administrativas y servicios auxi...,Ambos sexos,Porcentaje,2022T1,5.3
4,N Actividades administrativas y servicios auxi...,Ambos sexos,Porcentaje,2021T4,5.4


In [146]:
df_porcentaje_2022T1 = df.loc[(df['Unidad'] == "Porcentaje") & (df['Periodo'] == "2022T1")].reset_index()
df_porcentaje_2022T1 = df_porcentaje_2022T1[["Rama de actividad CNAE 2009", "Sexo", "Total"]]

df_absoluto_2022T1 = df.loc[(df['Unidad'] == "Valor absoluto") & (df['Periodo'] == "2022T1")].reset_index()
df_absoluto_2022T1 = df_absoluto_2022T1[["Rama de actividad CNAE 2009", "Sexo", "Total"]]

In [147]:
df_porcentaje_2022T1.head()

,Rama de actividad CNAE 2009,Sexo,Total
0,N Actividades administrativas y servicios auxi...,Ambos sexos,5.3
1,N Actividades administrativas y servicios auxi...,Hombres,4.4
2,N Actividades administrativas y servicios auxi...,Mujeres,6.3
3,"R Actividades artísticas, recreativas y de ent...",Ambos sexos,2.0
4,"R Actividades artísticas, recreativas y de ent...",Hombres,2.2


In [154]:
# Sort the data to be plotted
categories_sorted = df[df['Sexo'] == "Ambos sexos"].sort_values("Total", ascending = False)['Rama de actividad CNAE 2009'].unique()
df1 = df_porcentaje_2022T1.set_index("Rama de actividad CNAE 2009")
df_porcentaje_2022T1_sorted = df1.loc[categories_sorted].reset_index()
df_porcentaje_2022T1_sorted.head(50)

,Rama de actividad CNAE 2009,Sexo,Total
0,Total,Ambos sexos,100.0
1,Total,Hombres,100.0
2,Total,Mujeres,100.0
3,"G Comercio al por mayor y al por menor, repara...",Ambos sexos,14.6
4,"G Comercio al por mayor y al por menor, repara...",Hombres,13.8
5,"G Comercio al por mayor y al por menor, repara...",Mujeres,15.6
6,C Industria manufacturera,Ambos sexos,12.1
7,C Industria manufacturera,Hombres,16.4
8,C Industria manufacturera,Mujeres,7.0
9,Q Actividades sanitarias y de servicios sociales,Ambos sexos,9.2


**OCUPADOS POR SEXO Y RAMA DE ACTIVIDAD (CNAE 2009)**

In [157]:
pd.pivot_table(df_porcentaje_2022T1_sorted, index = "Rama de actividad CNAE 2009", columns = ['Sexo']).loc[categories_sorted]

Total                
Sexo                                               Ambos sexos Hombres Mujeres
Rama de actividad CNAE 2009                                                   
Total                                                    100.0   100.0   100.0
G Comercio al por mayor y al por menor, reparac...        14.6    13.8    15.6
C Industria manufacturera                                 12.1    16.4     7.0
Q Actividades sanitarias y de servicios sociales           9.2     3.7    15.6
I Hostelería                                               7.5     6.3     8.8
P Educación                                                7.6     4.4    11.4
O Administración Pública y defensa, Seguridad S...         6.8     7.1     6.5
F Construcción                                             6.6    11.0     1.4
M Actividades profesionales, científicas y técn...         5.6     5.1     6.1
N Actividades administrativas y servicios auxil...         5.3     4.4     6.3
H Transporte y almacenamiento                              5.4     7.9     2.4
A Agricultura, ganadería, silvicultura y pesca             4.1     5.8     2.1
J Información y comunicaciones                             3.7     4.9     2.3
T Actividades de los hogares como empleadores d...         2.7     0.5     5.2
K Actividades financieras y de seguros                     2.3     2.0     2.5
S Otros servicios                                          2.5     1.6     3.4
R Actividades artísticas, recreativas y de entr...         2.0     2.2     1.7
L Actividades inmobiliarias                                0.8     0.8     0.9
E Suministro de agua, actividades de saneamient...         0.7     1.1     0.2
D Suministro de energía eléctrica, gas, vapor y...         0.5     0.6     0.4
B Industrias extractivas                                   0.1     0.2     0.0
U Actividades de organizaciones y organismos ex...         0.0     0.0     0.0

In [142]:
df_porcentaje_2022T1_sorted.head(50)

,Rama de actividad CNAE 2009,Sexo,Total
0,Total,Ambos sexos,100.0
1,Total,Hombres,100.0
2,Total,Mujeres,100.0
3,G,Ambos sexos,14.6
4,G,Hombres,13.8
5,G,Mujeres,15.6
6,C,Ambos sexos,12.1
7,C,Hombres,16.4
8,C,Mujeres,7.0
9,Q,Ambos sexos,9.2


In [93]:
rama_actividad_encode = {}
rama_actividad_decode = {}
for name in df["Rama de actividad CNAE 2009"].unique():
    if name == "Total": 
        rama_actividad_encode["Total"] = "Total"
        rama_actividad_decode["Total"] = "Total"

    else: 
        rama_actividad_encode[name] = name[0]
        rama_actividad_decode[name[0]] = name

rama_actividad_encode

{'N Actividades administrativas y servicios auxiliares': 'N',
 'R Actividades artísticas, recreativas y de entretenimiento': 'R',
 'T Actividades de los hogares como empleadores de personal doméstico, actividades de los hogares como productores de bienes y servicios para uso propio': 'T',
 'U Actividades de organizaciones y organismos extraterritoriales': 'U',
 'K Actividades financieras y de seguros': 'K',
 'L Actividades inmobiliarias': 'L',
 'M Actividades profesionales, científicas y técnicas': 'M',
 'Q Actividades sanitarias y de servicios sociales': 'Q',
 'O Administración Pública y defensa, Seguridad Social obligatoria': 'O',
 'A Agricultura, ganadería, silvicultura y pesca': 'A',
 'G Comercio al por mayor y al por menor, reparación de vehículos de motor y motocicletas': 'G',
 'F Construcción': 'F',
 'P Educación': 'P',
 'I Hostelería': 'I',
 'C Industria manufacturera': 'C',
 'B Industrias extractivas': 'B',
 'J Información y comunicaciones': 'J',
 'S Otros servicios': 'S',
 'E

In [94]:
df["Rama de actividad CNAE 2009"] = df["Rama de actividad CNAE 2009"].apply(lambda x: rama_actividad_encode[x])
df.head()

,Rama de actividad CNAE 2009,Sexo,Unidad,Periodo,Total
0,N,Ambos sexos,Valor absoluto,2022T1,1061.5
1,N,Ambos sexos,Valor absoluto,2021T4,1098.9
2,N,Ambos sexos,Valor absoluto,2021T3,1103.7
3,N,Ambos sexos,Porcentaje,2022T1,5.3
4,N,Ambos sexos,Porcentaje,2021T4,5.4
